In [1]:
import pandas as pd
import numpy as np

In [2]:
#read files
monday = pd.read_csv('monday.csv', sep=';', parse_dates=True, index_col=0)
tuesday = pd.read_csv('tuesday.csv', sep=';', parse_dates=True, index_col=0)
wednesday = pd.read_csv('wednesday.csv', sep=';',parse_dates=True, index_col=0)
thursday = pd.read_csv('thursday.csv', sep=';', parse_dates=True, index_col=0)
friday = pd.read_csv('friday.csv', sep=';', parse_dates=True, index_col=0)

FileNotFoundError: [Errno 2] No such file or directory: 'monday.csv'

In [ ]:
monday

In [ ]:
tuesday

In [ ]:
monday[monday['customer_no'] == 1]

In [ ]:
#add day column and create one dataframe with all data
monday['day'] = 'monday'
tuesday['day'] = 'tuesday'
wednesday['day'] = 'wednesday'
thursday['day'] = 'thursday'
friday['day'] = 'friday'



df =  pd.concat([monday, tuesday, wednesday, thursday, friday])
df.sort_values('customer_no').head()

In [ ]:
#create customer id column for unique keys
df['customer_id']= df.day + '_' + df.customer_no.astype(str)

In [ ]:
df

In [ ]:
df[df.location=='checkout']

In [ ]:
#tasks : - we need to consider every minute (each minute one row!)
# --> groupby('customer_no').resample('1Min').ffill()
#       - not every customer left the supermarket (dairy diaries)  
# --> checkout time = 9.59pm    
#       - we need the transition info in the table (from where to where are cust going in probabilities)
# --> transition matrix : a. shift(1) the location column to get the "from" column
#                         b. fillna('checkout') [there is one NaN in the first row]
#.                        c. pd.crosstab(df['from'], df['location'], normalize='index') 
#[split it if you want certain windows of time]

In [ ]:
# Get all customers
total_customers = set(df['customer_id']) # use set-method to get a list with customers numbers
print("Total customers: ", len(total_customers))

In [ ]:
# Display the number of customers at checkout over time
customers_checkout = set(df.loc[df['location'] == 'checkout', 'customer_id']) # use set-method to get a list with customers numbers
print("Checked out customers: ", len(customers_checkout))

In [ ]:
# Display the number of customers that didn't checkout
customers_no_checkout = total_customers.difference(customers_checkout) # user difference-method to get a list with the customers numbers
print("Not checked out customers: ", customers_no_checkout)

In [ ]:
def add_checkout(df, customer_ids):
    for c_id in customer_ids:
        date= df[df['customer_id']== c_id].index.day[0]
        #c_no = df.loc[df['customer_id']== c_id, 'customer_no'].values[0]
        day = c_id.split("_")[0]
        c_no = c_id.split("_")[1]
        df_tmp = pd.DataFrame(data=[[c_no, 'checkout', day, c_id]], index=[pd.to_datetime(f'2019-09-{date} 21:59:00')], columns=['customer_no', 'location', 'day','customer_id'])
        df = pd.concat([df, df_tmp])
    return df    

In [ ]:
len(df.customer_id.unique())

In [ ]:
df.sort_values('customer_id')

In [ ]:
df[df.location=='checkout']

In [ ]:
df

In [ ]:
filled_data = []

checkout_customers = set(df.loc[df['location'] == 'checkout', 'customer_id'])
no_checkout_customers = list(total_customers.difference(checkout_customers))

#for i in enumerate(df.customer_id):
#checkout = 
df = add_checkout(df, no_checkout_customers)
df.tail()
#filled_data.append(checkout)

In [ ]:
# Get all customers
total_customers = set(df['customer_id']) # use set-method to get a list with customers numbers
print("Total customers: ", len(total_customers))
# Display the number of customers at checkout over time
customers_checkout = set(df.loc[df['location'] == 'checkout', 'customer_id']) # use set-method to get a list with customers numbers
print("Checked out customers: ", len(customers_checkout))
customers_no_checkout = total_customers.difference(customers_checkout) # user difference-method to get a list with the customers numbers
print("Not checked out customers: ", customers_no_checkout)

In [ ]:
df

In [ ]:
set(df.loc[df['location'] == 'checkout', 'customer_id'])


In [ ]:
for i, day in enumerate(df.day):
    print(add_checkout(day, no_checkout_customers[i], date[i], weekdays[i]))

In [ ]:
df

In [ ]:
# df = df.groupby('customer_no').resample('1Min').ffill()
# df.head()

In [ ]:
# IF we want to reset index
#df.reset_index(level='timestamp', inplace= True)
#df.drop('customer_no', axis=1, inplace=True)


In [ ]:
#for day in days ?

In [ ]:
#add 'from' column and replace 
test['from'].replace({'checkout':'entry'}, inplace = True)

In [ ]:
pd.crosstab(test('from'), test('location'), normalize='index')